# HSC-ACT Offset Distribution

With the HSC-ACT crossmatch catalog, one can fit the offsets distribution.

In [ ]:
## Import packages
import numpy as np
from scipy.optimize import minimize
from scipy import integrate

from astropy.table import Table
from astropy.io import fits, ascii

import matplotlib.pyplot as plt

## Get list of offsets

In [ ]:
# read in the crossmatch catalog
cm_catalog = Table.read('HSC_ACT_crossmatch.fits', hdu=1)
# read in "Cause of miscentering" column
cause_of_misc = cm_catalog['Cause of miscentering']

In [ ]:
# create a flag indicating if a miscentered cluster should be excluded in the fitted distribution
shape = (len(cm_catalog))

reasons_for_exclusion = {'Artifact', 'Misidentified CG', 'Deblending', 'False match', 
                         'False signal', 'Star'}
exclude_from_distro = [(cause in reasons_for_exclusion) for cause in cause_of_misc]
exclude_from_distro = np.array(exclude_from_distro)

In [ ]:
offsets_fiducial = cm_catalog['Offset (Mpc)']

exclude_clusters = True
if exclude_clusters:
    offsets = offsets_fiducial[~exclude_from_distro]
else:
    offsets = offsets_fiducial

## Create offsets model

In [ ]:
def prob_dens_well_centered(x, sigma1):
    '''
    A model for the offset distribution for well-centered clusters. See
    the documentation for offset_model for a description.
    
    
    ARGUMENTS
    ---------
    x: physical offset
    sigma1: characteristic offset for well-centered clusters
    
    '''
    
    return (x / sigma1**2) * np.exp(-x**2 / (2*sigma1**2))

def prob_dens_mis_centered(x, sigma2):
    '''
    A model for the offset distribution for miscentered clusters. See
    the documentation for offset_model for a description.
    
    ARGUMENTS
    ---------
    x: physical offset
    sigma2: characteristic offset for miscentered clusters
    
    '''
    
    return (x / sigma2**2) * np.exp(-x**2 / (2*sigma2**2))

def prob_dens_linear(x, slope=0.5):
    '''
    A linear component of the offsets model that represents random 
    superpositions of ACT and HSC clusters. 
    
    
    '''
    
    return slope*x

def offset_model(x, f_cen, sigma1, sigma2):
    '''
    
    A two-component model for the centering offset distribution. Returns 
    the probability density at a given offset. All arguments are in Mpc.
    
    This is equation 9 in the CAMIRA catalog paper (Oguri et al. 2018).
    Both the well-centered population and the miscentered population are
    modeled with a Rayleigh distribution.

    ARGUMENTS
    ---------
    x: physical offset
    f_cen: fraction of well-centered clusters
    sigma1: characteristic offset for well-centered clusters
    sigma2: characteristic offset for miscentered clusters
    
    '''
    
    prob_dens_well = prob_dens_well_centered(x, sigma1)
    prob_dens_mis = prob_dens_mis_centered(x, sigma2)
    
    return f_cen * prob_dens_well + (1 - f_cen) * prob_dens_mis

def offset_model_fixed_sig1(x, f_cen, sigma2, sigma1=0.1):
    '''
    
    Same as offset_model, except sigma1 is a fixed value. 
    All arguments are in Mpc.
    

    ARGUMENTS
    ---------
    x: physical offset
    f_cen: fraction of well-centered clusters
    sigma2: characteristic offset for miscentered clusters
    sigma1: characteristic offset for well-centered clusters; default
    value is 0.1 Mpc
    
    '''
    
    prob_dens_well = prob_dens_well_centered(x, sigma1)
    prob_dens_mis = prob_dens_mis_centered(x, sigma2)
    
    return f_cen * prob_dens_well + (1 - f_cen) * prob_dens_mis

def offset_model_fixed_sig1_3comp(x, f_cen, sigma2, slope, sigma1=0.15):
    '''
    
    3-component model of the offsets distribution. The well-centered 
    population is modeled with a Rayleigh distribution with (fixed) sigma1,
    the miscentered population is a Rayleigh distribution with sigma2, and
    the random superposition population is linear.
    
    All arguments are in Mpc.
    

    ARGUMENTS
    ---------
    x: physical offset
    f_cen: fraction of well-centered clusters
    slope: characteristic slope of random superposition population
    sigma2: characteristic offset for miscentered clusters
    sigma1: characteristic offset for well-centered clusters; default
    value is 0.1 Mpc
    
    '''
    
    prob_dens_well = prob_dens_well_centered(x, sigma1)
    prob_dens_mis = prob_dens_mis_centered(x, sigma2)
    prob_dens_random = prob_dens_linear(x, slope)
    
    return f_cen * prob_dens_well + (1 - f_cen) * prob_dens_mis + prob_dens_random

def objective_fn(params, offsets):
    '''
    
    Given the model parameters and the set of offsets, compute
    the objective as a log likelihood.
    
    ARGUMENTS
    ----------
    params: tunable parameters in offset_model
    offsets: set of physical offsets (Mpc; same as offset_model)
    
    '''
    
    f_cen, sig1, sig2 = params
    probs = offset_model(offsets, f_cen, sig1, sig2)
    
    obj = -np.sum(np.log(probs)) # goal: *minimize* log likelihood
    return obj

def objective_fn_fixed_sig1(params, offsets, fixed_sig1):
    '''
    
    Given the parameters for the 2-component model (with fixed sig1) and 
    the set of offsets, compute the objective as a log likelihood.
    
    ARGUMENTS
    ----------
    params: tunable parameters in offset_model
    offsets: set of physical offsets (Mpc; same as offset_model)
    fixed_sig1: fixed value of sigma1
    
    '''
    
    f_cen, sig2 = params
    
    probs = offset_model_fixed_sig1(offsets, f_cen, sig2, sigma1=fixed_sig1)
    
    obj = -np.sum(np.log(probs)) # goal: *minimize* log likelihood
    return obj

def objective_fn_fixed_sig1_3comp(params, offsets, fixed_sig1):
    '''
    
    Given the parameters for the 3-component model and the set of offsets, 
    compute the objective as a log likelihood.
    
    ARGUMENTS
    ----------
    params: tunable parameters in offset_model
    offsets: set of physical offsets (Mpc; same as offset_model)
    fixed_sig1: fixed value of sigma1
    
    '''
    
    f_cen, sig2, slope = params
    
    probs = offset_model_fixed_sig1_3comp(offsets, f_cen, sig2, slope, sigma1=fixed_sig1)
    
    obj = -np.sum(np.log(probs)) # goal: *minimize* log likelihood
    return obj

## Fit distribution to model

In [ ]:
use_fixed_sig1 = True
use_3comp = False

if use_fixed_sig1:
    fixed_sig1 = 0.15
    sig1_best = fixed_sig1
    
    if use_3comp:
        initial_guess = (0.75, 0.17, 0.1)
        best_guess = minimize(fun=objective_fn_fixed_sig1_3comp, x0=initial_guess, \
                        args=(offsets, fixed_sig1), method='L-BFGS-B', \
                              bounds=((0., 1), (0.01, 1.), (0., 0.2)))
        f_best, sig2_best, slope_best = best_guess.x
    else:
        initial_guess = (0.75, 0.17)
        best_guess = minimize(fun=objective_fn_fixed_sig1, x0=initial_guess, \
                    args=(offsets, fixed_sig1), method='L-BFGS-B', \
                          bounds=((0., 1), (0.01, 1.)))
        f_best, sig2_best = best_guess.x
else:
    initial_guess = (0.75, 0.15, 0.35)
    best_guess = minimize(fun=objective_fn, x0=initial_guess, args=(offsets), \
                         method='L-BFGS-B', bounds=((0., 1), (1e-4, 0.3), (0.1, 1.)))
    f_best, sig1_best, sig2_best = best_guess.x

print(best_guess)

## Plot distribution and model together

In [ ]:
## set global font size
plt.rcParams.update({'font.size':32})

## Plot offset distribution (cross-match)
fig = plt.figure(figsize=(25,14))
fig.patch.set_facecolor('white')

use_title = False
if use_title:
    plot_title = f"ACT-HSC offset distribution"
    plt.suptitle(plot_title, fontsize=30, y=0.95)

    plot_subtitle = f'n = {len(offsets)}'
    if exclude_clusters:
        plot_subtitle += '; clusters with an identifiable, non-astrophysical source of miscentering are excluded.'
    plt.title(plot_subtitle, fontsize=24, pad=10)

## Data sample
num_bins = 20
max_offset = np.max(offsets)
bins = np.linspace(0, max_offset, num=num_bins)
hist = plt.hist(offsets, bins=bins, \
                label=f"Offset distribution", density=True)
plt.xlabel("Physical offset (Mpc)", labelpad=10)
plt.ylabel("Normalized number of clusters", labelpad=10)

## Model
x_axis = (hist[1][:-1] + hist[1][1:]) / 2 # midpoints of histogram bins
# plot full model
norm_factor_full = 1 / \
        integrate.quad(offset_model, 0, max_offset, args=(f_best, sig1_best, sig2_best))[0]

# plot the full model
if use_3comp:
    model = norm_factor_full*offset_model_fixed_sig1_3comp(x_axis, f_best, sig2_best,\
                                                        slope_best, fixed_sig1)
else:
    model = norm_factor_full*offset_model_fixed_sig1(x_axis, f_best, sig2_best,\
                                                        fixed_sig1)
    
plt.plot(x_axis, model, \
    label=f"Full model (f_cen={np.round(f_best,3)})", color='black', linewidth=5)

# plot the model's components separately
if use_fixed_sig1:
    plt.plot(x_axis, norm_factor_full*f_best*prob_dens_well_centered(x_axis, sig1_best),
         label=f"Well-centered component ($\sigma_1$={np.round(sig1_best,3)}, fixed)", \
         color='purple', linestyle='--', linewidth=4)
else:
    plt.plot(x_axis, norm_factor_full*f_best*prob_dens_well_centered(x_axis, sig1_best),
         label=f"Well-centered component ($\sigma_1$={np.round(sig1_best,3)})", \
         color='purple', linestyle='--', linewidth=4)

plt.plot(x_axis, norm_factor_full*(1-f_best)*prob_dens_mis_centered(x_axis, sig2_best), 
         label=f"Miscentered component ($\sigma_2$={np.round(sig2_best,3)})", \
         color='red', linestyle=':', linewidth=5)

if use_3comp:
    plt.plot(x_axis, prob_dens_linear(x_axis, slope_best), 
         label=f"Linear component (slope={np.round(slope_best,3)})", \
         color='green', linestyle='--', linewidth=5)

plt.legend()
plt.savefig("Figures/MNRAS_Paper_Figures/Model/cleaned_distro.png", bbox_inches='tight')
plt.show()

## Analyze fit

We assign the labels of "well-centered" and "miscentered" to the cross-match clusters. 


Below, the cutoff for well-centered vs. miscentered is determined. For example: Assume $f_{cen}=0.75$. If a cluster's centering offset is above the 75th percentile of all offsets in the sample, then it is miscentered; otherwise, it is well-centered.

In [ ]:
well_centered_cutoff = np.percentile(offsets, 100*f_best)
print(f'well_centered_cutoff: {np.round(well_centered_cutoff, 3)} Mpc')
is_well_centered = [(offset < well_centered_cutoff) for offset in offsets]
is_well_centered = np.asarray(is_well_centered)

Below, we determine the number of clusters whose offset falls between 331 and 373 kpc. (The 331 kpc came from the fiducial crossmatch, and the 373 kpc comes from this "cleaned" crossmatch.)

In [ ]:
# offsets_sorted = np.sort(offsets)
cutoff1 = 0.331
cutoff2 = 0.373

between_cutoffs = (offsets > cutoff1) & (offsets < cutoff2)
#offsets[between_cutoffs]

# examine miscentering causes of the clusters in this offset range
between_cutoffs_fiducial = (offsets_fiducial > cutoff1) & (offsets_fiducial < cutoff2)
# print clusters in cleaned sample in this offset range
cm_catalog[between_cutoffs_fiducial & ~exclude_from_distro]

In [ ]:
# examine miscentered clusters in cleaned sample
is_miscentered_cleaned = (offsets_fiducial > cutoff2) & ~exclude_from_distro
cm_catalog[is_miscentered_cleaned]